<center>
<h3>Universidade Federal do Rio Grande do Sul (UFRGS)<br>
Programa de Pós-Graduação em Engenharia Civil (PPGEC)<br>
PEC00144 - Métodos Experimentais em Engenharia Civil</h3>

<h1>Trabalho 2</h1>

<h3> Aluno: <a href="https://github.com/dutitello/">Eduarto Pagnussat Titello</a> </h3>

</center>


# Apresentação

Refazendo o exemplo do professor.

In [59]:
# Importando módulos
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
%config InlineBackend.figure_format = 'svg' # Muda backend do jupyter para SVG ;)
import jupyter2latex as j2l # Uma maneira que encontrei para tabelas ficarem ok (github.com/dutitello/Jupyter2Latex)

# Importando todas grandezas
DimData = pd.read_excel('../resources/DimData.xlsx', 
                         index_col  =  0,
                         sheet_name = 'DimData')

#j2l.df2table(DimData)

O problema aqui é a flecha de uma viga biapoiada retangular de propriedades:
- $L = 10m$
- $B \times H = 0.2 \times 0.5 m$
- $A = 0.1 m^2$
- $I=0.002083 m^4$
- $\rho = 2500 kg/m^3$
- $\mu_L = 250 kg/m^3$
- $E_c = 30 \times 10^9 N/m^2$
- $E_c \times I = 62.5 \times 10^6 Nm^2$

Sendo:
$$ w_{max} = \frac{5}{384}\frac{qL^4}{EI} $$

Considerando a carga aplicada $q$ como peso próprio: $q=\mu_L \times G = 250 kg/m \times 9.81 m/s^2 = 2452.5 N/m$, logo a flecha máxima para o peso próprio é:
$$ w_{max} = \frac{5}{384}\frac{2452.5N/m \times 10m^4}{62.5 \times 10^6 Nm^2} = 0.00511 m = 5.11 mm$$


Adotando escalas de: (formato mod.reduzido : mod.real)
- dimensões: 1m:10m 
- gravidade: 1G:1G
- módulo E: 71GPa:30GPa
- densidade $\rho$: 2.7g/cm³:2.5g/cm³

é necessária a adoção de uma base que considere tais grandezas, lembrando que E e $\sigma$ tem mesmo formato de unidades:

In [60]:
# Unidades fundamentais originais 
LMT = ['L', 'M', 'T']

# Novas unidades fundamentais
ABC = ['L', 'a', 'σ']

# Importa matriz dimensional de ABC na base LMT
base = DimData.loc[ABC, LMT]
j2l.df2table(base, 'ABC na base LMT')

# Inverte base de unidades de LMT para ABC
base_i = pd.DataFrame(np.linalg.inv(base), index=LMT, columns=ABC)
j2l.df2table(base_i, 'LMT na base ABC')

,L,M,T
L,1,0,0
a,1,0,-2
σ,-1,1,-2


,L,a,σ
L,1.0,0.0,0.0
M,2.0,-1.0,1.0
T,0.5,-0.5,-0.0


Matrizes dimensioanais dos parametros à serem testados no modelo reduzido

In [62]:
par = ['F', 'q', 'L', 'EI', 'μL', 'a']
npar = len(par)

DMat1 = DimData.loc[par, LMT]
j2l.df2table(DMat1, 'Matriz D na base LMT')

DMat2 = np.matmul(DMat1, base_i)
DMat2.rename(columns=dict(zip(LMT,ABC)), inplace=True)
j2l.df2table(DMat2, 'Matriz D na base ABC')

,L,M,T
F,1,1,-2
q,0,1,-2
L,1,0,0
EI,3,1,-2
μL,-1,1,0
a,1,0,-2


,L,a,σ
F,2.0,0.0,1.0
q,1.0,0.0,1.0
L,1.0,0.0,0.0
EI,4.0,0.0,1.0
μL,1.0,-1.0,1.0
a,0.0,1.0,0.0


Definição da escala:

In [29]:
λ_L    =  1/10         # length scale for the reduced model
λ_a    =  1/1          # acceleration remains unchanged (same gravity)
λ_σ    =  71/30        # model built with aluminum instead of concrete

scales = np.array([λ_L, λ_a, λ_σ])
scales = np.tile(scales, (npar, 1))

print(scales)

[[0.1        1.         2.36666667]
 [0.1        1.         2.36666667]
 [0.1        1.         2.36666667]
 [0.1        1.         2.36666667]
 [0.1        1.         2.36666667]
 [0.1        1.         2.36666667]]


Valores de escala dos parametros

In [32]:
λs = np.prod(scales**DMat2, axis=1)

ScaleFactors = pd.DataFrame({'λ': λs, '1/λ': 1/λs}, index=par)

j2l.df2table(ScaleFactors)

,λ,1/λ
F,0.023667,42.253521
q,0.236667,4.225352
L,0.100000,10.000000
EI,0.000237,4225.352113
μL,0.236667,4.225352
a,1.000000,1.000000


Com isso é possível determinar as propriedades do modelo reduzido, como por exemplo: $(EI)_m = \lambda_{EI} . E_c . I $.

In [35]:
EI = 62.5e6
EIm = ScaleFactors.loc['EI', 'λ'] * EI
print(f"EI do modelo reduzido deve ser {EIm:.3E} Nm²")

EI do modelo reduzido deve ser 1.479E+04 Nm²


Adotando uma seção retangular de altura 5cm, a largura é definida de forma que: 
$$(EI)_m = E_m \times \frac{b_m h_m^3}{12}$$
logo:
$$ b_m = \frac{12(EI)_m}{E_mh_m^3} $$

In [44]:
Em = 71e9
hm = 0.05
bm = 12*EIm/(Em*hm**3)
print(f'Largura da viga no modelo reduzido: {100*bm:.2f}cm')

Largura da viga no modelo reduzido: 2.00cm


Com isso o peso própio do modelo é:

In [55]:
qm = 2700*hm*bm*9.81
qm_esp = ScaleFactors.loc['q', 'λ']*250*9.81
print(f'Peso próprio do modelo: {qm:.3f}N/m')
print(f'Peso próprio esperado do modelo: {qm_esp:.3f}N/m')

Peso próprio do modelo: 26.487N/m
Peso próprio esperado do modelo: 580.425N/m


Logo o modelo precisa ser muito mais pesado para que sejá valido... mesmo assim vamos testar a hipótese:

In [56]:
wm = 5/384*qm_esp*1**4/(EIm)
print(f'Flecha do modelo reduzido: {1000*wm:.3f}mm')

Flecha do modelo reduzido: 0.511mm


Ok, a flecha do modelo reduzido é 1/10 da flecha do modelo real, o que era esperado!

Como o modelo é muito pesado é impraticável sua construção. Uma maneira alternativa é introduzir uma escala em EI, **forçando o uso de um pefil existente**, ficando com uma base: L, a, EI.